In [3]:
import torch
device = torch.device('cuda')
torch.cuda.is_available()

True

In [4]:
from kp_api import KeypointExtractor

c:\Users\48113164\AppData\Local\miniconda3\envs\torch-gpu\lib\site-packages\mmengine\optim\optimizer\zero_optimizer.py:11: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import \


In [5]:
kp_cfg_path='configs/wholebody_2d_keypoint/rtmpose/cocktail14/rtmw-l_8xb320-270e_cocktail14-384x288.py'
kp_ckpt_path='rtmw-dw-x-l_simcc-cocktail14_270e-384x288-20231122.pth'

In [6]:
import os
import sys
import cv2
import subprocess
import tempfile
import numpy as np
import pandas as pd

In [7]:
kp_extractor=KeypointExtractor(kp_cfg_path, kp_ckpt_path, device=device)

Loads checkpoint by local backend from path: rtmw-dw-x-l_simcc-cocktail14_270e-384x288-20231122.pth


c:\Users\48113164\AppData\Local\miniconda3\envs\torch-gpu\lib\site-packages\mmengine\runner\checkpoint.py:347: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = tor

Keypoint extractor model loaded successfully.


In [ ]:
import os
import tarfile
import numpy as np
from pathlib import Path
import time
import subprocess
remote_files_processed=[]
PROGRESS_FILE = Path("processed_files.txt")

rclone_remote_root = "drive:MyDrive/Kinetics400"
# Directorio fuente (donde están los .tar.gz)
root_targz_local = Path("k400_targz_temp")

# Directorio de destino para los VIDEOS (Requisito 1)
root_videos = Path("k400_videos")

# Directorio de destino para los KEYPOINTS (Requisito 2)
root_keypoints_out = Path("k400_keypoints")

# Subcarpetas a procesar (igual que en tu script de Bash)
sub_folders = ['train', 'val', 'test', 'replacement']

# Extensiones de video a buscar después de extraer
VIDEO_EXTENSIONS = {'.mp4', '.avi', '.mkv', '.webm', '.mov'}
def load_processed_files():
    """Carga la lista de archivos ya procesados desde disco."""
    if PROGRESS_FILE.exists():
        with open(PROGRESS_FILE, "r", encoding="utf-8") as f:
            return set(line.strip() for line in f if line.strip())
    return set()

def save_processed_file(file_path: str):
    """Agrega un archivo procesado al registro."""
    with open(PROGRESS_FILE, "a", encoding="utf-8") as f:
        f.write(file_path + "\n")
def get_remote_file_list(remote_path: str) -> list:
    """
    Usa 'rclone lsf' para listar archivos .tar.gz y .tgz en un directorio remoto.
    """
    print(f"📡 Obteniendo lista de archivos remotos de: {remote_path}")
    command = ["rclone", "lsf", "--files-only", remote_path]
    
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True, encoding='utf-8')
        all_files = result.stdout.splitlines()
        # Filtrar solo los tarballs
        tar_files = [f for f in all_files if f.endswith('.tar.gz') or f.endswith('.tgz')]
        print(f"  -> Encontrados {len(tar_files)} archivos tarball.")
        return tar_files
    except subprocess.CalledProcessError as e:
        print(f"❌ ERROR al listar archivos con rclone: {e.stderr}")
        return []
    except FileNotFoundError:
        print("❌ ERROR: 'rclone' no se encuentra. ¿Está instalado y en tu PATH?")
        return []
def download_file(remote_file_path: str, local_file_path: Path) -> bool:
    """
    Descarga un SOLO archivo usando 'rclone copyto'.
    """
    print(f"  ⬇️ Descargando: {remote_file_path}...")
    # Asegurarse de que el directorio local exista
    local_file_path.parent.mkdir(parents=True, exist_ok=True) 
    
    command = ["rclone", "copyto", remote_file_path, str(local_file_path)]
    
    try:
        # Usamos capture_output para no llenar la consola con el log de rclone
        subprocess.run(command, check=True, capture_output=True)
        print("  -> Descarga completa.")
        return True
    except subprocess.CalledProcessError as e:
        print(f"    ❌ ERROR al descargar {remote_file_path}: {e.stderr}")
        return False

def process_single_tarball(local_tar_path: Path, videos_dir: Path, keypoints_dir: Path):
    """
    Procesa UN solo archivo .tar.gz:
    1. Extrae videos.
    2. Procesa videos -> .npy.
    3. BORRA videos.
    """
    print(f"  🗜️ Extrayendo: {local_tar_path.name} a {videos_dir}")
    try:
        with tarfile.open(local_tar_path, "r:gz") as tar:
            tar.extractall(path=videos_dir)
    except Exception as e:
        print(f"    ⚠️ AVISO: Error al extraer {local_tar_path.name}: {e}. Omitiendo.")
        return

    # --- Procesar Videos Extraídos ---
    print(f"  🤖 Procesando videos extraídos...")
    extracted_videos = []
    for ext in VIDEO_EXTENSIONS:
        # Busca solo en la carpeta de videos, no recursivo
        extracted_videos.extend(videos_dir.glob(f'*{ext}'))
    
    if not extracted_videos:
        print("    -> No se encontraron videos en este tarball.")
        return

    for video_path in extracted_videos:
        npy_filename = video_path.with_suffix('.npy').name
        npy_path = keypoints_dir / npy_filename

        if npy_path.exists():
            # print(f"    -> Ya existe, omitiendo: {npy_path.name}")
            pass # Omitir en silencio
        else:
            try:
                # 2. PROCESAR
                keypoints_array = kp_extractor.extract_from_video(video_path)
                np.save(npy_path, keypoints_array)
            except Exception as e:
                print(f"    ERROR al procesar el video {video_path.name}: {e}")
        
        # 3. BORRAR VIDEO (¡Importante para ahorrar espacio!)
        try:
            os.remove(video_path)
        except Exception as e:
            print(f"      AVISO: No se pudo borrar el video {video_path.name}: {e}")
    
    print(f"  ✅ Videos procesados y borrados.")

if __name__ == "__main__":
    print("Iniciando script de procesamiento sostenible (uno por uno)...")
    start_time = time.time()
    processed_files = load_processed_files()

    for folder in sub_folders:
        print(f"\n{'='*20} \n📂 Carpeta Remota: {folder} \n{'='*20}")
        remote_folder_path = f"{rclone_remote_root}/{folder}"
        local_targz_dir = root_targz_local / folder
        local_videos_dir = root_videos / folder
        local_keypoints_dir = root_keypoints_out / folder
        
        local_targz_dir.mkdir(parents=True, exist_ok=True)
        local_videos_dir.mkdir(parents=True, exist_ok=True)
        local_keypoints_dir.mkdir(parents=True, exist_ok=True)

        remote_files = get_remote_file_list(remote_folder_path)
        if not remote_files:
            print(f"  No se encontraron archivos en {remote_folder_path}. Omitiendo.")
            continue

        total_files = len(remote_files)
        print(f"  Se procesarán {total_files} archivos...")

        for i, filename in enumerate(remote_files, 1):
            remote_file_path = f"{remote_folder_path}/{filename}"

            # 🔹 SALTAR si ya se procesó antes
            if remote_file_path in processed_files:
                print(f"  ⚡ Ya procesado: {filename}, omitiendo.")
                continue

            print(f"\n  --- Procesando archivo {i}/{total_files}: {filename} ---")
            local_tar_path = local_targz_dir / filename
            
            if not download_file(remote_file_path, local_tar_path):
                print("  Error en descarga. Omitiendo este archivo.")
                continue
            
            process_single_tarball(local_tar_path, local_videos_dir, local_keypoints_dir)
            
            try:
                os.remove(local_tar_path)
                print(f"  🗑️ .tar.gz temporal borrado: {local_tar_path.name}")
            except Exception as e:
                print(f"    AVISO: No se pudo borrar {local_tar_path.name}: {e}")
            
            # ✅ Registrar archivo como procesado
            save_processed_file(remote_file_path)
            processed_files.add(remote_file_path)

    end_time = time.time()
    print(f"\n{'='*20}\n🎉 ¡Proceso completo! \n{'='*20}")
    print(f"Directorio de Keypoints: {root_keypoints_out.resolve()}")
    print(f"Tiempo total: {end_time - start_time:.2f} segundos.")


Iniciando script de procesamiento sostenible (uno por uno)...

📂 Carpeta Remota: train 
📡 Obteniendo lista de archivos remotos de: drive:MyDrive/Kinetics400/train
  -> Encontrados 242 archivos tarball.
  Se procesarán 242 archivos...

  --- Procesando archivo 1/242: part_0.tar.gz ---
  ⬇️ Descargando: drive:MyDrive/Kinetics400/train/part_0.tar.gz...
